In [1]:
import io
import json
import sqlite3 as sql
import pandas as pd
import os

def query(query_string, connection):
    return pd.read_sql_query(query_string, connection)

In [2]:
file = open('owl.sql', 'w+')

In [3]:
json_file = open('matches.json')
data = json.load(json_file)['content']
json_file.close()

In [4]:
match_schema = (
    'CREATE TABLE [MATCHES] ( '
    'MATCH_ID INT, '
    'TEAM_ONE_ID INT, '
    'TEAM_TWO_ID INT, '
    'TEAM_ONE_SCORE INT, '
    'TEAM_TWO_SCORE INT, '
    'WINNER_ID INT, '
    'TIMESTAMP DATETIME, '
    'PRIMARY KEY (MATCH_ID)'
    ');' 
)

In [5]:
file.write(match_schema + '\n\n')

174

In [6]:
for match in data:
    insert = (
        'INSERT INTO MATCHES '
        '(MATCH_ID, TEAM_ONE_ID, TEAM_TWO_ID, '
        'TEAM_ONE_SCORE, TEAM_TWO_SCORE, '
        'WINNER_ID, TIMESTAMP)'
        'VALUES (%d, %d, %d, %d, %d, %d, %d);'
        % (match['id'], match['competitors'][0]['id'],
          match['competitors'][1]['id'],
          match['scores'][0]['value'],
          match['scores'][1]['value'],
          match['winner']['id'],
          match['actualStartDate'])
    )
    file.write(insert+'\n')

In [7]:
#connection = sql.connect('test.db')

In [8]:
#cursor = connection.cursor()
#db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
#database = db.read()
#cursor.executescript(database)

In [9]:
game_schema = (
    '\nCREATE TABLE [GAMES] ('
    'GAME_ID INT, '
    'MATCH_ID INT, '
    'MAP TEXT, '
    'PRIMARY KEY (GAME_ID)'
    ');\n\n'
)

file.write(game_schema)

85

In [10]:
for match in data:
    for game in match['games']:
        try:
            map_name = str(game['attributes']['map']).replace('-', ' ')
            insert = (
                'INSERT INTO GAMES (GAME_ID, MATCH_ID, MAP) VALUES '
                '(%d, %d, \'%s\');'
                % (game['id'], match['id'], map_name)
            )
            file.write(insert+'\n')
        except:
            print('Error')

Error
Error


In [11]:
#try:
#    connection = sql.connect('test.db')
#    cursor = connection.cursor()
#    db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
#    database = db.read()
#    cursor.executescript(database)
#except Exception as e:
#    print(e)
#    os.remove('test.db')

In [12]:
teams_schema = (
    '\nCREATE TABLE [TEAMS] ('
    'TEAM_ID INT, '
    'TEAM_NAME TEXT, '
    'TEAM_LOCATION TEXT, '
    'PRIMARY KEY (TEAM_ID)'
    ');\n\n'
)

file.write(teams_schema)

97

In [13]:
team_list = []
for match in data:
    for competitor in match['competitors']:
        if competitor['id'] not in team_list:
            team_list.append(competitor['id'])
            insert = (
                    'INSERT INTO TEAMS (TEAM_ID, TEAM_NAME, TEAM_LOCATION) VALUES '
                    '(%d, \'%s\', \'%s\');'
                    % (competitor['id'], 
                       competitor['name'], 
                       competitor['homeLocation'])
                )
            file.write(insert+'\n')

In [14]:
#try:
#    connection = sql.connect('test.db')
#    cursor = connection.cursor()
#    db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
#    database = db.read()
#    cursor.executescript(database)
#except Exception as e:
#    print(e)
#    os.remove('test.db')

In [15]:
teams_schema = (
    '\nCREATE TABLE [PLAYERS] ('
    'COMPETITOR_ID INT,'
    'TEAM_ID INT, '
    'HANDLE TEXT, '
    'ROLE TEXT, '
    'NUMBER INT,'
    'NAME TEXT,'
    'PRIMARY KEY (COMPETITOR_ID)'
    ');\n\n'
)

file.write(teams_schema)

132

In [17]:
team_list = []
player_list = []
for match in data:
    for competitor in match['competitors']:
        if competitor['id'] not in team_list:
            team_list.append(competitor['id'])
            for player in competitor['players']:
                if player['player']['id'] not in player_list:
                    player_list.append(player['player']['id'])
                    insert = (
                            'INSERT INTO PLAYERS (COMPETITOR_ID, TEAM_ID, HANDLE, ROLE, NUMBER, NAME) VALUES '
                            '(%d, %d, \'%s\', \'%s\', %d, \'%s\');'
                            % (player['player']['id'], 
                               player['team']['id'], 
                               player['player']['name'],
                               player['player']['attributes']['role'],
                               player['player']['attributes']['player_number'],
                               ' '.join([player['player']['givenName'], player['player']['familyName']]))
                        )
                    file.write(insert+'\n')

In [18]:
file.close()

In [19]:
try:
    connection = sql.connect('test.db')
    cursor = connection.cursor()
    db = io.open('owl.sql', 'r', encoding='ISO-8859-1')
    database = db.read()
    cursor.executescript(database)
except Exception as e:
    print(e)
    os.remove('test.db')

In [21]:
query('select h from players natural join teams', connection)

,COMPETITOR_ID,TEAM_ID,HANDLE,ROLE,NUMBER,NAME,TEAM_NAME,TEAM_LOCATION
0,8218,4409,Marve1,tank,11,Min Seo Hwang,Seoul Dynasty,Seoul
1,8831,4409,FITS,offense,16,Dong Eon Kim,Seoul Dynasty,Seoul
2,9279,4409,ILLICIT,offense,21,Je Min Park,Seoul Dynasty,Seoul
3,4098,4409,ZUNBA,tank,5,Joonhyeok Kim,Seoul Dynasty,Seoul
4,4104,4409,tobi,support,7,Jinmo Yang,Seoul Dynasty,Seoul
...,...,...,...,...,...,...,...,...
169,4652,7697,ArK,support,2,Yeonjun Hong,Washington Justice,"Washington, DC"
170,8784,7697,Corey,offense,11,Corey Nigra,Washington Justice,"Washington, DC"
171,8783,7697,Stratus,offense,27,Ethan Yankel,Washington Justice,"Washington, DC"
172,11102,7697,LullSiSH,tank,15,Lukas Wiklund,Washington Justice,"Washington, DC"
